In [1]:
import time
import csv
import json
import random
import time
import Levenshtein
import wikitextparser as wtp
from wiki_dump_reader import iterate
from tqdm import tqdm
from statistics import mean

from extraction_croissement import *

In [4]:
start_loading_data_time = time.time()

# Loading des données sauvegardées dans la mémoire ram
g_book_ADP = rdflib.Graph()
g_author_ADP = rdflib.Graph()
ADP_book_graph = g_book_ADP.parse("../Graphes/grapheADPLivres.rdf")
ADP_author_graph = g_author_ADP.parse("../Graphes/grapheADPAuteurs.rdf")
ADP_books = get_ADP_books(g_book_ADP, g_author_ADP)
ADP_loading_time = time.time()
print("ADP_loading_time: ", ADP_loading_time - start_loading_data_time)

g_item_DL = rdflib.Graph()
book_graph_DL = g_item_DL.parse("../Graphes/grapheDepotLegal.rdf")
DL_books = get_depot_legal_book(g_item_DL)
DL_loading_time = time.time()
print("DL_loading_time: ", DL_loading_time - ADP_loading_time)

# g_item_ILE = rdflib.Graph()
# item_graph_ILE = g_item_ILE.parse("../Graphes/grapheILE.rdf")
# ILE_books = get_ILE_book(g_item_ILE)
# ILE_loading_time = time.time()
# print("ILE_loading time: ", ILE_loading_time - DL_loading_time)

ILE_books = get_ILE_book_csv()
ILE_loading_time = time.time()
print("ILE_loading time: ", ILE_loading_time - DL_loading_time)

books_Hurtubise_file = open("./Hurtubise/Exportation-Hurtubise.csv", "r", encoding='ISO-8859-1')
csv_reader = csv.DictReader(books_Hurtubise_file, delimiter=',', fieldnames=[
    "Editeur", "ISBN Papier", "ISBN PDF", "ISBN epub", "Titre", "Sous - titre", "Titre de la serie",
    "Contributeurs", "Contributeur(premier)", "Langue", "Langue Origine", "Resume", "Nombre de pages",
    "Date de parution", "Annee de parution", "Sujet  THEMA principal", "Sujet THEMA",
    "Quantificateur Georaphique", "Quantificateur de langue", "Quantificateur Historique", "Niveau soclaire FR",
    "Niveau scolaire QC", "Cycle scolaire FR", "Niveau de lecture", "Echele CECR", "Quantificateur d'interet",
    "Quantificateur d'age", "Quantificateur de style", "Classification Editoriale", "Mots cles"

])
Hurtubise_books = get_Hurtubise_books(csv_reader)
books_Hurtubise_file.close()

authors_ILE_file = open("./ILE/auteurs_ILE_comma_separated.csv", 'r', encoding='ISO-8859-1')
csv_reader = csv.DictReader(authors_ILE_file, delimiter=',', fieldnames=[
    'uri', 'nom', 'bio', 'genres', 'site', 'pseudonyme'])
authors_ILE = [x for x in csv_reader]
authors_ILE_file.close()

authors_wikidata_file = open("./Wikidata/ecrivains_wikidata_comma_separated.csv", 'r', encoding='ISO-8859-1')
csv_reader = csv.DictReader(authors_wikidata_file, delimiter=',', fieldnames=[
    'nom', 'uri'])
authors_wikidata = [x for x in csv_reader]
authors_wikidata_file.close()

authors_DBpedia_file = open("./DBpedia/ecrivains_dbpedia_fr.txt", "r", encoding='ISO-8859-1')
csv_reader = csv.DictReader(authors_DBpedia_file, delimiter=';', fieldnames=[
    'uri', 'nom'])
authors_DBpedia = [x for x in csv_reader]
authors_DBpedia_file.close()

babelioJsonBooks = open("./Babelio/babelio_livres.json", "r")
Babelio_books = get_Babelio_books(json.load(babelioJsonBooks))
babelioJsonBooks.close()

babelioJsonAuthor = open("./Babelio/babelio_auteurs.json", "r")
Babelio_authors = get_Babelio_books(json.load(babelioJsonAuthor))
babelioJsonAuthor.close()

loading_data_time = time.time()
print("loading_data_time: ", loading_data_time - start_loading_data_time)

all_books = ADP_books + ILE_books + Hurtubise_books + Babelio_books + DL_books
random.shuffle(all_books)

with open('all_books.json', 'w') as outfile:
    json.dump(all_books, outfile)

ADP_loading_time:  11.377426385879517
DL_loading_time:  45.36634039878845
ILE_loading time:  3.089451313018799
loading_data_time:  60.115187644958496


In [5]:
with open('all_books.json', 'r') as input_file:
    all_books = json.load(input_file)
author_ls = {}
print('recoupement des auteurs')
for book in tqdm(all_books, total=len(all_books)):
    if len(book['author']) == 1 and isinstance(book['author'][0], list):
        book['author'] = book['author'][0]
    for author in book['author']:
        if author.replace('"', '') not in author_ls:
            author_ls[author.replace('"', '')] = [book]
        else:
            author_ls[author.replace('"', '')].append(book)

with open('authors_ls.json', 'w') as outfile:
    json.dump(author_ls, outfile)

100%|██████████| 102985/102985 [00:00<00:00, 867987.28it/s]


recoupement des auteurs


In [4]:
# def parse_Info(str):
#     result = re.search(r'\{\{Infobox (Ouvrage|Livre)(.|\n)+\}\}', str)
#     info_box_str = result.group()
#     nb_bracket = 0
#     for index, caractere in enumerate(info_box_str):
#         if caractere == '{':
#             nb_bracket += 1
#         elif caractere == '}':
#             nb_bracket -= 1
#         if nb_bracket == 0:
#             return info_box_str[:index]
#
# def get_info(text):
#     raw_infos = parse_Info(text)
#     infos = {}
#     for info in raw_infos.split('\n')[1:-1]:
#         test = info.split('=')
#         key = info.split('=')[0]
#         value = info.split('=')[1]
#         infos[key] = value
#     return infos

# couple_books = []
# with open('wikipedia/fr_dumps_wikipedia_books.json') as json_file:
#     data = json.load(json_file)
#     for title, text in tqdm(data.items(), total=len(data.items())):
#         for book in all_books:
#             dist_titre = Levenshtein.distance(book['title'], normalize(title))
#             if dist_titre < max(1, min(len(book['title']), len(title)) / 4):
#                 print("titre DB: ", book['title'], "titre de wiki:", title)
#                 new_couple = {
#                         'titre DB': book['title'],
#                         'titre wiki': title,
#                         'book DB': book,
#                     }
#                 try:
#                     new_couple['book wiki'] = get_info(text)
#                 except:
#                     new_couple['book wiki'] = text
#                 couple_books.append(new_couple)
#
# with open('couple_wiki_books.json', 'w') as outfile:
#     json.dump(couple_books, outfile)

# confirmed_couple = []
# unconfirmed_couple = []
# for couple in tqdm(couple_books, total=len(couple_books)):
#     if isinstance(couple['book wiki'], dict):
#         keys = list(couple['book wiki'].keys())
#         for key in keys:
#             couple['book wiki'][normalize(key)] = normalize(couple['book wiki'][key])
#         if 'auteur' in couple['book wiki'] and couple['book wiki']['auteur']:
#             correspondance = False
#             for author in couple['book DB']['author']:
#                 test = (couple['book wiki']['auteur'], author)
#                 dist_auteur = Levenshtein.distance(couple['book wiki']['auteur'], author)
#                 if dist_auteur < max(1, min(len(couple['book wiki']['auteur']), len(author)) / 4):
#                     confirmed_couple.append(couple)
#                     correspondance = True
#                     break
#             if not correspondance:
#                 unconfirmed_couple.append(couple)
#     else:
#         # print('a faire a la main: titre DB=', couple['titre DB'],' titre wiki=', couple['titre wiki'], ' wiki text=', couple['book wiki'][:200])
#         pass
# print('nombre de correspondance: ', len(confirmed_couple))
# print('nombre de distinct: ', len(unconfirmed_couple))
# print('confirmed_couple:')
# print(json.dumps(confirmed_couple, indent=2))
# print('unconfirmed_couple:')
# print(json.dumps(unconfirmed_couple, indent=2))

In [2]:
def format_list(page):
    page = re.sub(r'\*\*', '*', page)
    page_serach_re = re.search(r'\{\{colonnes\|.*?\|\n', page, re.DOTALL)
    while page_serach_re:
        nb_bracket = 0
        start_index = page_serach_re.span()[0]
        for index, caractere in enumerate(page[start_index:]):
            if caractere == '{':
                test_str = page[start_index:start_index + index + 1]
                nb_bracket += 1
            elif caractere == '}':
                test_str = page[start_index:start_index + index + 1]
                nb_bracket -= 1
            if nb_bracket == 0:
                test_7_1 = page[:start_index + index - 1]
                test_7_2 = page[start_index + index + 2:]
                page = page[:start_index + index - 1] + page[start_index + index + 2:]
                page = re.sub(r'\{\{colonnes\|.*?\|\n', '', page)
                break
        page_serach_re = re.search(r'\{\{colonnes\|.*?\|\n', page, re.DOTALL)
    return page

def split_info_box(str):
    nb_bracket = 0
    start_index = str.find("{{Infobox")
    for index, caractere in enumerate(str[start_index:]):
        if caractere == '{':
            test_str = str[start_index:start_index + index + 1]
            nb_bracket += 1
        elif caractere == '}':
            test_str = str[start_index:start_index + index + 1]
            nb_bracket -= 1
        if nb_bracket == 0:
            return str[:start_index + index + 1], str[start_index + index + 2:]
    return "", str
def get_info(raw_infos):
    if not raw_infos:
        return text
    infos = {}
    for info in re.split(r"\n\||\n\}", raw_infos, flags=re.DOTALL)[1:-1]:
        if len(info.split('=')) > 1:
            key, value = info.split('=')[0], info.split('=')[1]
            infos[key] = value
    return infos

def get_list_from_section(section):
    if section.get_lists():
        return [re.sub(r"\'\'", "", item) for list in section.get_lists() for item in list.items]
    else:
        return None


def parse_section(section, title_ls, level):

    section_dict = {'content': section.contents,
                    'list': get_list_from_section(section),
                    'sub_section': {}}
    if len(section.sections) > 2:
        sub_sections = section.sections[2:]
        for sub_sect in sub_sections:
            if sub_sect.level == level + 1:
                if sub_sect.title:
                    if sub_sect.title not in title_ls:
                        title_ls.append(sub_sect.title)
                    section_dict['sub_section'][sub_sect.title] = parse_section(sub_sect, title_ls, sub_sect.level)
                else:
                    section_dict['sub_section']['untitled'] = parse_section(sub_sect, title_ls, sub_sect.level)

    return section_dict

with open('wikipedia/fr_dumps_wikipedia_writers.json', 'r') as json_file:
    data = json.load(json_file)

wiki_authors = []
title_dict = {}
counter = 1
for title, text in tqdm(data.items()):
    try:

        normalised_title = normalize(title)
        new_author = {
                'name_raw': title,
                'name': normalised_title,
                'raw_text': text,
            }

        info_box, page = split_info_box(text)

        new_author['info_box'] = get_info(info_box)
        parsed_page = wtp.parse(format_list(page))
        sections_infos = {'untitled':[]}
        title_dict_couple = []
        for section in parsed_page.sections:
            if section.level <= 2:
                if section.title:
                    if section.title not in title_dict:
                        title_dict_couple.append(section.title)
                    sections_infos[section.title] = parse_section(section, title_dict_couple, section.level)
                else:
                    sections_infos['untitled'].append(section.contents)
        new_author['parsed_text'] = sections_infos
        wiki_authors.append(new_author)
    except:
        print("ca c'est mal passé....")
        print(title)
        print(text)

print("titre des sections: ", sorted(title_dict.items(), key=lambda item: item[1], reverse=True))

with open('wikipedia/parsed_writers_dump.json', 'w') as outfile:
    json.dump(wiki_authors, outfile)

100%|██████████| 40625/40625 [03:36<00:00, 187.81it/s] 


ca c'est mal passé....
José Tolentino de Mendonça
{{Infobox Prélat catholique
| nom                = José Tolentino de Mendonça
| titre              = Cardinal
| image              = Cardeal De Mendonça.jpg
| taille image       = 200
| légende            = Le cardinal De Mendonça, le 6 octobre 2019
| nom de naissance   = <!-- wikidata : Property:P1477 -->
| date de naissance  = <!-- wikidata : Property:P569 -->
| lieu de naissance  = Machico (Madère)
| date de décès      = <!-- wikidata : Property:P570 -->
| lieu de décès      = <!-- wikidata : Property:P20 -->
| ordre              = <!-- wikidata : Property:P611 -->
| profession solennelle =
| ordination         = 28 juillet 1990
| consécration       = 28 juillet 2018 <br>par le [[cardinal (religion)|card.]] [[Manuel do Nascimento Clemente|do Nascimento Clemente]]
| dernier titre      = [[Archives secrètes du Vatican|Archiviste]] et [[Bibliothèque apostolique vaticane|bibliothécaire de la Sainte Église romaine]]
| création           =

In [4]:
couple_writers_4 = []
couple_writers_6 = []
couple_writers_8 = []

data_base_stats_4 = {}
data_base_stats_6 = {}
data_base_stats_8 = {}

found_once = 0
count = 0
count_freq = {}
count_info = 0

with open('wikipedia/parsed_writers_dump.json', 'r') as input_file:
    wiki_authors = json.load(input_file)

with open('authors_ls.json', 'r') as input_file:
    author_ls = json.load(input_file)

for author_DB in tqdm(author_ls, total=len(author_ls)):
    found = False
    count_freq_i = 0
    for author_wiki in wiki_authors:
        dist_auteur = Levenshtein.distance(author_DB, author_wiki['name'])

        if dist_auteur < max(1, min(len(author_DB), len(author_wiki['name'])) / 4):
            new_couple = {
                        'author DB': author_DB,
                        'author wiki': author_wiki['name_raw'],
                        'book DB': author_ls[author_DB],
                        'info_wiki': author_wiki
                    }
            count += 1
            count_freq_i += 1
            found = True

            if dist_auteur < max(1, min(len(author_DB), len(author_wiki['name'])) / 8):
                    couple_writers_8.append(new_couple)
                    for book in author_ls[author_DB]:
                        try:
                            data_base_stats_8[book['data_base']] += 1
                        except:
                            data_base_stats_8[book['data_base']] = 1
            elif dist_auteur < max(1, min(len(author_DB), len(author_wiki['name'])) / 6):
                couple_writers_6.append(new_couple)
                for book in author_ls[author_DB]:
                    try:
                        data_base_stats_6[book['data_base']] += 1
                    except:
                        data_base_stats_6[book['data_base']] = 1
            else:
                couple_writers_4.append(new_couple)
                for book in author_ls[author_DB]:
                    try:
                        data_base_stats_4[book['data_base']] += 1
                    except:
                        data_base_stats_4[book['data_base']] = 1
    if found:
        found_once += 1
        count_freq[author_DB] = count_freq_i
    # if count_freq_i > 10:
    #     print("compte superieur a 10: ", author_DB)

print("nombre d'autheurs en commun: ", found_once, " soit ", found_once * 100 / len(list(author_ls)), "%")
print("nombre d'autheurs reconnus: ", count, " soit ", count * 100 / len(list(author_ls)), "%")
print("Nombre d'information catégorisée: ", count_info, " soit ", count_info * 100 / len(list(author_ls)), "%")
print("Frequence de reconnaissance: ", count_freq)

print("repartition dans les bases de données: ")
print('1/8: ', json.dumps(data_base_stats_8))
print('1/6: ', json.dumps(data_base_stats_6))
print('1/4: ', json.dumps(data_base_stats_4))

with open('wikipedia/couple_authors_wikipedia_4.json', 'w') as outfile:
    json.dump(couple_writers_4, outfile)

with open('wikipedia/couple_authors_wikipedia_6.json', 'w') as outfile:
    json.dump(couple_writers_6, outfile)

with open('wikipedia/couple_authors_wikipedia_8.json', 'w') as outfile:
    json.dump(couple_writers_8, outfile)


100%|██████████| 20059/20059 [18:36<00:00, 17.96it/s]


nombre d'autheurs en commun:  5551  soit  27.673363577446533 %
nombre d'autheurs reconnus:  10700  soit  53.342639214317764 %
Nombre d'information catégorisée:  0  soit  0.0 %
Frequence de reconnaissance:  {'louis hemon': 3, 'etienne lalonde': 1, 'michel-marc bouchard': 1, 'alainbergeron': 2, 'regine robin': 1, 'wilma alexander': 1, 'david lonergan': 1, 'sophie berube': 3, 'marthe gagnon-thibaudeau': 1, 'monique plante': 1, 'andre brochu': 1, 'paul-marie lapointe': 1, 'francois chartier': 5, 'sergio kokis': 1, 'julie stanton': 1, 'anique poitras': 1, 'valerie menard': 2, 'roger roge': 1, 'don delillo': 1, 'bertrand gervais': 1, 'frederick durand': 3, 'marie-celie agnant': 1, 'madeleine ouellette-michalska': 1, 'yvon jean': 1, 'adolphe-basile routhier': 1, 'michel tremblay': 1, 'cecile gagnon': 1, 'denis diderot': 1, 'lionel groulx': 1, 'laure conan': 1, 'gervais pomerleau': 1, 'jean lemieux': 1, 'sylvain riviere': 1, 'marie-francine hebert': 1, 'marie laberge': 2, 'naim kattan': 1, 'ro

In [ ]:
def extract_title_from_list(list):
    titles = []
    for item in [item.replace('\n', '') for item in list]:
        added = False
        meta_data_re = re.search(r"\{\{(Ouvrage|Écrit).*\}\}", item)
        if meta_data_re:
            meta_data = meta_data_re.group()
            title_re = re.search(r'(?<=titre=).*?(?=\|)', meta_data)
            subtitle_re = re.search(r'(?<=sous-titre=).*?(?=\|)', meta_data)
            if title_re or subtitle_re:
                added = True
                if title_re and subtitle_re:
                    titles.append(normalize(title_re.group() + ' ' + subtitle_re.group()))
                elif title_re:
                    titles.append(normalize(title_re.group()))
                elif subtitle_re:
                    titles.append(normalize(subtitle_re.group()))
        if not added:
            n = 1
            while n:
                item, n = re.subn(r"\{\{.*\}\}", '', item)
            n = 1
            while n:
                item, n = re.subn(r"<.+?>.*?</.+?>", '', item)
            brackets_re = re.search(r'(?<=\[\[).*?(?=\]\])', item)
            while brackets_re:
                test = brackets_re.group().split('\|')
                item = re.sub(r"\[\[.*?\]\]", brackets_re.group().split('|')[-1], item, count=1)
                brackets_re = re.search(r'(?<=\[\[).*?(?=\]\])', item)
            item = remove_text_between_parentheses(item)
            item = re.sub(r'<ref .+?\\>', '', item)
            item = re.sub(r'^\s*?\d{4}', '', item)
            item = re.sub(r'^-\s', '', item)
            item = re.sub(r'^:', '', item)
            if normalize(item.split(',')[0]):
                titles.append(normalize(item.split(',')[0]))
    return titles

from statistics import mean

famous = ['Michel Tremblay', 'Anne Hébert', 'Gabrielle Roy', 'Marie Cardinal', 'Réjean Ducharme',
                'Jacques Ferron', 'Victor-Lévy Beaulieu', 'Marcel Dubé', 'Yves Thériault', 'Jacques Poulin',
                'André Langevin']

with open('wikipedia/couple_authors_wikipedia_8.json', 'r') as couples_file:
    couple_writers = json.load(couples_file)

with open('authors_ls.json', 'r') as input_file:
    author_ls = json.load(input_file)

section_title_list = {}
data_base_stats = {}

couples_confirmed = []
couples_infirmed = []
couples_unsure = []

count_work_dict = {}
count_checked_by_titles = 0
count_no_titles = 0
count_differentiate_by_titles = 0

for couple in couple_writers:
    confirmed_by_title_match = False
    confirmed_by_title_match_re = False
    book_titles = set()
    book_titles_raw = set()
    for book in author_ls[couple['author DB']]:
        try:
            data_base_stats[book['data_base']] += 1
        except:
            data_base_stats[book['data_base']] = 1
    info = couple['info_wiki']['parsed_text']
    for title in info:
        if 'list' in info[title] and info[title]['list']:
            if title not in section_title_list:
                section_title_list[title] = 1
            else:
                section_title_list[title] += 1
            if re.search(r'bibliographie|Œuvres|Œuvre|théâtre|poésie|nouvelles|essais|publications|romans|roman|Littérature|Ouvrages|Filmographie', title, re.IGNORECASE):
                count_work_dict[couple['author DB']] = len(info[title]['list'])
                book_titles.update(extract_title_from_list(info[title]['list']))
                book_titles_raw.update(info[title]['list'])
    try:
        book_titles_DB = [book['title'] for book in couple['book DB']]
        for title in book_titles_DB:
            for title_wiki in book_titles:
                dist_titles = Levenshtein.distance(title, title_wiki)
                if dist_titles < max(1, min(len(title), len(title_wiki)) / 4):
                    confirmed_by_title_match = True
                    break
        if not confirmed_by_title_match:
            for title in book_titles_DB:
                for title_wiki_raw in book_titles_raw:
                    if re.search(normalize(title), normalize(title_wiki_raw)):
                        confirmed_by_title_match_re = True
                        break

    except:
        pass

    if (not confirmed_by_title_match) and (not confirmed_by_title_match_re):
        if book_titles:
            count_differentiate_by_titles += 1
            couples_infirmed.append(couple)
        else:
            count_no_titles += 1
            couples_unsure.append(couple)
    else:
        count_checked_by_titles += 1
        couples_confirmed.append(couple)

    couple['confirmed_by_title_match'] = confirmed_by_title_match
    couple['confirmed_by_title_match_re'] = confirmed_by_title_match_re

# print(json.dumps(sorted(section_title_list.items(), key=lambda item: item[1], reverse=True)[:20], indent=1))
# print(mean(count_work_dict.values()))
print("Répartition des couples par oeuvres dans les bases de données")
print(json.dumps(data_base_stats, indent=1))

# for couple in couple_writers[:100]:
#     print('Auteur DB: ', couple['author DB'], " " * (40 - len(couple['author DB'])),' Auteur wiki ' ,couple['author wiki'], 'confirmé par comparaison de titres: ', couple['confirmed_by_title_match'], 'confirmé par regex: ', couple['confirmed_by_title_match_re'])
print('nombre de couple: ', len(couple_writers))
print('nombre de couples confirmés par titres: ', count_checked_by_titles)
print('nombre de couples infirmés par titres: ', count_differentiate_by_titles)
print('nombre de couples sans titres de wikipedia: ', count_no_titles)

with open('wikipedia/couple_authors_wikipedia_confirmed.json', 'w') as outfile:
    json.dump(couples_confirmed, outfile)

with open('wikipedia/couple_authors_wikipedia_infirmed.json', 'w') as outfile:
    json.dump(couples_infirmed, outfile)

with open('wikipedia/couple_authors_wikipedia_unsure.json', 'w') as outfile:
    json.dump(couples_unsure, outfile)


In [10]:
def extract_title_from_list(list):
    titles = []
    for item in [item.replace('\n', '') for item in list]:
        added = False
        meta_data_re = re.search(r"\{\{(Ouvrage|Écrit).*\}\}", item)
        if meta_data_re:
            meta_data = meta_data_re.group()
            title_re = re.search(r'(?<=titre=).*?(?=\|)', meta_data)
            subtitle_re = re.search(r'(?<=sous-titre=).*?(?=\|)', meta_data)
            if title_re or subtitle_re:
                added = True
                if title_re and subtitle_re:
                    titles.append(normalize(title_re.group() + ' ' + subtitle_re.group()))
                elif title_re:
                    titles.append(normalize(title_re.group()))
                elif subtitle_re:
                    titles.append(normalize(subtitle_re.group()))
        if not added:
            n = 1
            while n:
                item, n = re.subn(r"\{\{.*\}\}", '', item)
            n = 1
            while n:
                item, n = re.subn(r"<.+?>.*?</.+?>", '', item)
            brackets_re = re.search(r'(?<=\[\[).*?(?=\]\])', item)
            while brackets_re:
                test = brackets_re.group().split('\|')
                item = re.sub(r"\[\[.*?\]\]", brackets_re.group().split('|')[-1], item, count=1)
                brackets_re = re.search(r'(?<=\[\[).*?(?=\]\])', item)
            item = remove_text_between_parentheses(item)
            item = re.sub(r'<ref .+?\\>', '', item)
            item = re.sub(r'^\s*?\d{4}', '', item)
            item = re.sub(r'^-\s', '', item)
            item = re.sub(r'^:', '', item)
            if normalize(item.split(',')[0]):
                titles.append(normalize(item.split(',')[0]))
    return titles

with open('wikipedia/couple_authors_wikipedia_8.json', 'r') as input_file:
    couples_writers = json.load(input_file)

count = 0
mean_count = []
section_title_list = {}
for couple in couples_writers:
    award_titles = set()
    info = couple['info_wiki']['parsed_text']
    for title in info:
        if 'list' in info[title] and info[title]['list']:
            if title not in section_title_list:
                section_title_list[title] = 1
            else:
                section_title_list[title] += 1
            if re.search(r'prix', title, re.IGNORECASE):
                awards = extract_title_from_list(info[title]['list'])
                award_titles.update(awards)

    if award_titles:
        count += 1
    mean_count.append(len(award_titles))
print("Nombre d'auteur dont on a récupéré des prix: ", count)
print("Nombre total de prix: ", sum(mean_count))
print("totals_price: ", totals_price)


Nombre d'auteur dont on a récupéré des prix:  335
Nombre total de prix:  2493
totals_price:  {'amina et le mamelouk blanc a ete finaliste du prix trillium', 'prix du public', 'distinction socrate', '- medaille marie-tremaine', 'prix alain-grandbois pour la devoration des fees', 'le prix du livre romantique en 1987 pour l amour baobab', 'sandhaas d or', '- grand prix du festival international de la poesie de la fondation les forges', 'prix heimito von doderer', 'selection prix renaudot pour vers le sud', 'mon oeil', 'en 1978', 'belle mort', '- nomination au prix aurora', 'prix gutenberg d or', 'selection de malabourg au prix orange du livre', 'prix mary higgins clark 2019 pour bone on bone ref name nom1', '- prix international de litterature', 'officier des arts et lettres 2017', 'prix victor-morin', 'bourse de la fondation cyril collard', 'prix barry 2001 du meilleur livre de poche pour the kidnapping of rosie dawn', 'prix d honneur', 'finaliste prix real-fillion du festival de la band

In [14]:
with open('wikipedia/couple_authors_wikipedia_8.json', 'r') as couples_file:
    wikipedia_couple_writers = json.load(couples_file)

with open('Wikidata/couple_authors_wikidata.json', 'r') as couples_file:
    wikidata_couple_writers = json.load(couples_file)
#diagramme de weinn database, wikipedia, wikidata

count = 0
couples_wikipedia_wikidata = []
for wikipedia_couple in wikipedia_couple_writers:
    for wikidata_couple in wikidata_couple_writers:
        if wikidata_couple['author DB'] == wikipedia_couple['author DB']:
            count += 1
print("Nombre d'auteurs en communs: ", count)

with open('couple_authors_wikipedia_wikidata.json', 'w') as outfile:
    json.dump(couples_wikipedia_wikidata, outfile)



Nombre d'auteurs en communs:  758
